SKRIPSI

# Import library

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk
import os

from nltk import ngrams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, validation_curve,RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from matplotlib_venn import venn2
from time import time
from joblib import dump, load
from sklearn.metrics import confusion_matrix
from sklearn import metrics

np.random.seed(0)
stopwords = nltk.corpus.stopwords.words('english')
poster_Stemmer = nltk.PorterStemmer()


# Baca file csv

In [2]:
#baca csv
data_train_amazon = pd.read_csv('dataset/Amazon_Train.csv')
data_test_amazon = pd.read_csv('dataset/Amazon_Test.csv')
data_train_imdb = pd.read_csv('dataset/IMDB_Train_1000.csv')
data_test_imdb = pd.read_csv('dataset/IMDB_Test.csv')

# Preprocessing 

In [3]:
data_Preprocessing_Amazon_Train = data_train_amazon.Sentimen.astype(str)
data_Preprocessing_Amazon_Train = data_Preprocessing_Amazon_Train.apply(lambda x: x.lower())
data_Preprocessing_Amazon_Train = data_Preprocessing_Amazon_Train.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_Amazon_Train = data_Preprocessing_Amazon_Train.apply(
                                    lambda x: x.translate(str.maketrans('','',string.punctuation)))  
data_Preprocessing_Amazon_Train = data_Preprocessing_Amazon_Train.apply(
                                    lambda x: " ".join(re.findall("[a-zA-Z]+", x)))
data_Preprocessing_IMDB_Train = data_train_imdb.Sentimen.astype(str)
data_Preprocessing_IMDB_Train = data_Preprocessing_IMDB_Train.apply(lambda x: x.lower())
data_Preprocessing_IMDB_Train = data_Preprocessing_IMDB_Train.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_IMDB_Train = data_Preprocessing_IMDB_Train.apply(
                                    lambda x: x.translate(str.maketrans('','',string.punctuation)))  
data_Preprocessing_IMDB_Train = data_Preprocessing_IMDB_Train.apply(
                                    lambda x: " ".join(re.findall("[a-zA-Z]+", x)))


In [4]:
data_Preprocessing_Amazon_Train.head()

0    buyer beware this is a selfpublished book and ...
1    the worst a complete waste of time typographic...
2    oh please i guess you have to be a romance nov...
3    awful beyond belief i feel i have to write to ...
4    another abysmal digital copy rather than scrat...
Name: Sentimen, dtype: object

In [5]:
data_Preprocessing_IMDB_Train.head()

0    story of a man who has unnatural feelings for ...
1    airport starts as a brand new luxury plane is ...
2    this film lacked something i couldnt put my fi...
3    sorry everyone i know this is supposed to be a...
4    when i was little my parents took me along to ...
Name: Sentimen, dtype: object

In [6]:
data_Preprocessing_Amazon_Test = data_test_amazon.Sentimen.astype(str)
data_Preprocessing_Amazon_Test = data_Preprocessing_Amazon_Test.apply(lambda x: x.lower())
data_Preprocessing_Amazon_Test = data_Preprocessing_Amazon_Test.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_Amazon_Test = data_Preprocessing_Amazon_Test.apply(lambda x: x.translate(
                                str.maketrans('','',string.punctuation)))  
data_Preprocessing_Amazon_Test = data_Preprocessing_Amazon_Test.apply(lambda x: " ".join(
                                re.findall("[a-zA-Z]+", x)))

data_Preprocessing_IMDB_Test = data_test_imdb.Sentimen.astype(str)
data_Preprocessing_IMDB_Test = data_Preprocessing_IMDB_Test.apply(lambda x: x.lower())
data_Preprocessing_IMDB_Test = data_Preprocessing_IMDB_Test.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_IMDB_Test = data_Preprocessing_IMDB_Test.apply(
                                    lambda x: x.translate(str.maketrans('','',string.punctuation)))  
data_Preprocessing_IMDB_Test = data_Preprocessing_IMDB_Test.apply(
                                    lambda x: " ".join(re.findall("[a-zA-Z]+", x)))

In [7]:
data_Preprocessing_Amazon_Test.head()

0    overly complicated being both a us history buf...
1    terrible disappointment parts dont fit my son ...
2    didnt hold upvery disappointed i bought this o...
3    gene hates jezebel i love jlj but this compila...
4    nice toy but my sixyearold loves space and to ...
Name: Sentimen, dtype: object

In [8]:
data_Preprocessing_IMDB_Test.head()

0    once again mr costner has dragged out a movie ...
1    first of all i would like to say that i am a f...
2    im a huge fan of both emily watson breaking th...
3    i was pulled into this movie early on much to ...
4    this tale of the upperclasses getting their co...
Name: Sentimen, dtype: object

In [9]:
Clean_Amazon_Sentiment_Train = data_Preprocessing_Amazon_Train
Clean_IMDB_Sentiment_Train = data_Preprocessing_IMDB_Train
Label_Amazon_Train = data_train_amazon['Label']
Label_IMDB_Train = data_train_imdb['Label']

Clean_Amazon_Sentiment_Test = data_Preprocessing_Amazon_Test
Clean_IMDB_Sentiment_Test = data_Preprocessing_IMDB_Test
Label_Amazon_Test = data_test_amazon['Label']
Label_IMDB_Test = data_test_imdb['Label']

# N-Gram (bigram dan trigram)

In [10]:
def bigram_trigram(Clean_Sentiment) :
    iterator = 0
    Vocabulary = []
    ngram_result = []
    while iterator < len(Clean_Sentiment) :
        ngram = 2
        sentence = Clean_Sentiment[iterator]
        vocab = list(ngrams(sentence, ngram))
        temp = vocab

        iterator2 = 0
        while iterator2 < len (vocab):
            if(vocab[iterator2] not in Vocabulary) :
                Vocabulary.append(vocab[iterator2])
            iterator2 = iterator2 + 1

        ngram = 3
        vocab = list(ngrams(sentence, ngram))
        temp = temp + vocab
        ngram_result.append(temp)
        iterator2 = 0
        while iterator2 < len (vocab):
            if(vocab[iterator2] not in Vocabulary) :
                Vocabulary.append(vocab[iterator2])
            iterator2 = iterator2 + 1
        iterator = iterator + 1
    
    return ngram_result,Vocabulary


In [11]:
ngram_amazon_train, Vocabulary_Amazon_Train = bigram_trigram(Clean_Amazon_Sentiment_Train)

In [12]:
ngram_amazon_test, Vocabulary_Amazon_Test = bigram_trigram(Clean_Amazon_Sentiment_Test)

In [13]:
ngram_imdb_train, Vocabulary_IMDB_Train = bigram_trigram(Clean_IMDB_Sentiment_Train)

In [14]:
ngram_imdb_test, Vocabulary_IMDB_Test = bigram_trigram(Clean_IMDB_Sentiment_Test)

# Me inisialisasi jumlah frekuensi Gram di Vocabulary

In [15]:
def dictionaryInitialize(Clean_Sentiment, Vocabulary, ngram, jenisdata) :
    wordDictionaryCount = []
    iterator = 0
    while iterator < len(Clean_Sentiment):
        wordDictionaryCount.append(dict.fromkeys(Vocabulary , 0))
        iterator = iterator + 1
    
    iterator = 0
    
    while iterator < len(Clean_Sentiment) :
        for gram in ngram[iterator] :
            if jenisdata == 'test' and gram not in Vocabulary :
                continue
            wordDictionaryCount[iterator][gram] += 1
        iterator = iterator + 1
    
    return wordDictionaryCount

In [16]:
WordDictionaryCount_Amazon_Train = dictionaryInitialize(Clean_Amazon_Sentiment_Train, Vocabulary_Amazon_Train, ngram_amazon_train,'train')

In [17]:
WordDictionaryCount_Amazon_Test = dictionaryInitialize(Clean_Amazon_Sentiment_Test, Vocabulary_Amazon_Train, ngram_amazon_test,'test')

In [18]:
WordDictionaryCount_IMDB_Test = dictionaryInitialize(Clean_IMDB_Sentiment_Test, Vocabulary_IMDB_Train, ngram_imdb_test,'test')

In [19]:
WordDictionaryCount_IMDB_Train = dictionaryInitialize(Clean_IMDB_Sentiment_Train, Vocabulary_IMDB_Train, ngram_imdb_train,'train')

In [20]:
pd.DataFrame([WordDictionaryCount_Amazon_Train[0],WordDictionaryCount_Amazon_Train[1]])

,"(b, u)","(u, y)","(y, e)","(e, r)","(r, )","( , b)","(b, e)","(e, w)","(w, a)","(a, r)",...,"(k, o, y)","(t, y, n)","(w, s, y)","(i, u, n)","(b, y, p)","(b, u, o)","( , u, e)","(u, e, m)","(a, b, w)","(m, u, )"
0,1,1,1,11,9,13,5,3,4,7,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,2,5,1,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [21]:
pd.DataFrame([WordDictionaryCount_IMDB_Train[0],WordDictionaryCount_IMDB_Train[1]])

,"(s, t)","(t, o)","(o, r)","(r, y)","(y, )","( , o)","(o, f)","(f, )","( , a)","(a, )",...,"(z, v, o)","(d, o, v)","(k, r, e)","(f, b, a)","(i, d, y)","(g, s, h)","(o, g, b)","(z, , y)","(k, y, t)","(e, e, g)"
0,7,5,6,2,10,9,5,5,12,9,...,0,0,0,0,0,0,0,0,0,0
1,25,19,47,5,33,65,25,29,81,28,...,0,0,0,0,0,0,0,0,0,0


## Menghitung nilai TF pada setiap data 

In [22]:
def computeFrequency(wordDict, ngram):
    freqword = {}
    ngramLength = len(ngram)
    for word, count in wordDict.items():
        freqword[word] = count
    return freqword

In [23]:
def getFrequencyTermPerData(wordDictionary, ngram) :
    CountFrequency = []
    iterator = 0
    while iterator < len(ngram):
        CountFrequency.append(computeFrequency(wordDictionary[iterator], ngram[iterator]))
        iterator = iterator + 1
    
    return CountFrequency

In [24]:
CountFrequency_Sentimen_Amazon_Train = getFrequencyTermPerData(WordDictionaryCount_Amazon_Train, ngram_amazon_train)
CountFrequency_Sentimen_IMDB_Train = getFrequencyTermPerData(WordDictionaryCount_IMDB_Train, ngram_imdb_train)
CountFrequency_Sentimen_Amazon_Test = getFrequencyTermPerData(WordDictionaryCount_Amazon_Test, ngram_amazon_test)
CountFrequency_Sentimen_IMDB_Test = getFrequencyTermPerData(WordDictionaryCount_IMDB_Test, ngram_imdb_test)

## Menghitung nilai IDF pada setiap data Amazon dan Yelp

In [25]:
def computeIDF(WordDict):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDict)
    
    idfDict = dict.fromkeys(WordDict[0].keys(), 0)
    for doc in WordDict:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        number_of_document_with_term_t_in_it[word] = float(val)
        idfDict[word] = np.log((1+N) / (1+val)) + 1
        
    return idfDict, number_of_document_with_term_t_in_it

In [26]:
IDF_Amazon, number_of_document_with_term_t_Amazon_Train = computeIDF(WordDictionaryCount_Amazon_Train)
IDF_IMDB, number_of_document_with_term_t_in_IMDB_Train = computeIDF(WordDictionaryCount_IMDB_Train)

In [27]:
def computeTFIDFnonScale(CountFrequency, idfs):
    tfidf = None
    tfidf = {}
    for word, val in CountFrequency.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [28]:
def computeTFIDF(tfidf_nonscaled) :
    tfidf_nonscaled = np.array(tfidf_nonscaled)
    tfidf_list = tfidf_nonscaled/sum(tfidf_nonscaled**2)**0.5
    return tfidf_list

In [29]:
def getTFIDF(wordDictionary, CountFrequency, IDF):
    TFIDF_non_scaled = None
    TFIDF_non_scaled = []
    iterator = 0
    while iterator < len(wordDictionary):
        TFIDF_non_scaled.append(computeTFIDFnonScale(CountFrequency[iterator], IDF))
        iterator = iterator + 1
    
    TFIDF = []
    iterator = 0
    while iterator < len(wordDictionary):
        tf_idf_list = list(TFIDF_non_scaled[iterator].values())
        TFIDF.append(computeTFIDF(tf_idf_list))
        iterator = iterator + 1
        
    return TFIDF

In [30]:
TFIDF_Amazon_Train = getTFIDF(WordDictionaryCount_Amazon_Train, CountFrequency_Sentimen_Amazon_Train, IDF_Amazon)

In [31]:
TFIDF_Amazon_Test = getTFIDF(WordDictionaryCount_Amazon_Test, CountFrequency_Sentimen_Amazon_Test, IDF_Amazon)

In [32]:
TFIDF_IMDB_Train = getTFIDF(WordDictionaryCount_IMDB_Train, CountFrequency_Sentimen_IMDB_Train, IDF_IMDB)

In [33]:
TFIDF_IMDB_Test = getTFIDF(WordDictionaryCount_IMDB_Test, CountFrequency_Sentimen_IMDB_Test, IDF_IMDB)

In [34]:
def toVectorizeList(TFIDF):
    iterator  = 0
    Vectorized = []
    while iterator < len(TFIDF):
        Vectorized.append(list(TFIDF[iterator]))
        iterator = iterator + 1
    return Vectorized

In [35]:
Vectorizer_Amazon_Train = toVectorizeList(TFIDF_Amazon_Train)
Vectorizer_Amazon_Test = toVectorizeList(TFIDF_Amazon_Test)
Vectorizer_IMDB_Train = toVectorizeList(TFIDF_IMDB_Train)
Vectorizer_IMDB_Test = toVectorizeList(TFIDF_IMDB_Test)

## Akurasi dan Waktu Train Test 

In [36]:
Waktu_Training = time()
RF_Classifier_Amazon = RandomForestClassifier(
        max_depth= 5, n_estimators = 800, random_state=42,
        bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Amazon.fit(Vectorizer_Amazon_Train, Label_Amazon_Train)
print(f"\nWaktu Training Amazon: {round(time()-Waktu_Training, 3)}s")


Waktu Training Amazon: 25.475s


In [37]:
Waktu_Predict_Train = time()
Skor_Train_Amazon = RF_Classifier_Amazon.score(Vectorizer_Amazon_Train, Label_Amazon_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Amazon = RF_Classifier_Amazon.score(Vectorizer_Amazon_Test, Label_Amazon_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Amazon : {}".format(Skor_Train_Amazon))
print("Skor Random Forest Test Amazon : {}".format(Skor_Test_Amazon))

waktu prediksi (train): 2.629s
waktu prediksi (test): 0.885s

Skor Random Forest Train Amazon : 0.8978666666666667
Skor Random Forest Test Amazon : 0.8184


In [38]:
Waktu_Training = time()
RF_Classifier_IMDB = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_IMDB.fit(Vectorizer_IMDB_Train, Label_IMDB_Train)
print(f"\nWaktu Training IMDB: {round(time()-Waktu_Training, 3)}s")


Waktu Training IMDB: 7.578s


In [39]:
Waktu_Predict_Train = time()
Skor_Train_IMDB = RF_Classifier_IMDB.score(Vectorizer_IMDB_Train, Label_IMDB_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_IMDB = RF_Classifier_IMDB.score(Vectorizer_IMDB_Test, Label_IMDB_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train : {}".format(Skor_Train_IMDB))
print("Skor Random Forest Test  : {}".format(Skor_Test_IMDB))
print("----------------------------------------------------\n\n")

RFClassifier_predict = RF_Classifier_IMDB.predict(Vectorizer_IMDB_Train)
Confusion_matrix = confusion_matrix(Label_IMDB_Train, RFClassifier_predict)
print(Confusion_matrix)
print(metrics.classification_report(Label_IMDB_Train, RFClassifier_predict))

print("----------------------------------------------------\n\n")
RFClassifier_predict = RF_Classifier_IMDB.predict(Vectorizer_IMDB_Test)
Confusion_matrix = confusion_matrix(Label_IMDB_Test, RFClassifier_predict)
print(Confusion_matrix)
print(metrics.classification_report(Label_IMDB_Test, RFClassifier_predict))

waktu prediksi (train): 0.655s
waktu prediksi (test): 0.801s

Skor Random Forest Train : 0.992
Skor Random Forest Test  : 0.7392
----------------------------------------------------


[[498   2]
 [  6 494]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       500
           1       1.00      0.99      0.99       500

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

----------------------------------------------------


[[455 170]
 [156 469]]
              precision    recall  f1-score   support

           0       0.74      0.73      0.74       625
           1       0.73      0.75      0.74       625

    accuracy                           0.74      1250
   macro avg       0.74      0.74      0.74      1250
weighted avg       0.74      0.74      0.74      1250



## Mencari Term-Term yang interseksi antara Amazon dan IMDB

In [40]:
Vocabulary_Intersection = list(set(
                            Vocabulary_IMDB_Train) & set (
                            Vocabulary_Amazon_Train))

In [41]:
print(len(Vocabulary_Intersection))

6634


In [42]:
def RecomputeIDF(WordDictAmazon, WordDictYelp, additional_term_value, Vocabulary_Intersection):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDictYelp) + len(WordDictAmazon)
    idfDict = dict.fromkeys(WordDictYelp[0].keys(), 0)
    for doc in WordDictYelp:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        if word in Vocabulary_Intersection :
            val = val + additional_term_value[word]
        idfDict[word] = np.log((1+N) / (1+val)) + 1
        
    return idfDict

In [43]:
New_IDF_IMDB = RecomputeIDF(WordDictionaryCount_Amazon_Train, 
                            WordDictionaryCount_IMDB_Train, 
                            number_of_document_with_term_t_Amazon_Train, 
                            Vocabulary_Intersection)

In [44]:
New_TFIDF_Train = getTFIDF(WordDictionaryCount_IMDB_Train, CountFrequency_Sentimen_IMDB_Train, New_IDF_IMDB)
New_TFIDF_Test = getTFIDF(WordDictionaryCount_IMDB_Test, CountFrequency_Sentimen_IMDB_Test, New_IDF_IMDB)

In [45]:
Vectorizer_TF_Train = toVectorizeList(New_TFIDF_Train)
Vectorizer_TF_Test = toVectorizeList(New_TFIDF_Test)

In [46]:
New_TFIDF_Train = Vectorizer_TF_Train
New_TFIDF_Test = Vectorizer_TF_Test

In [47]:
Waktu_Training = time()
New_RF_Classifier_Yelp = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, 
                                        max_features = 'auto')
New_RF_Classifier_Yelp.fit(New_TFIDF_Train, Label_IMDB_Train)
print(f"\nWaktu Training: {round(time()-Waktu_Training, 3)}s")


Waktu Training: 7.751s


In [48]:
Waktu_Predict_Train = time()
Skor_Train_IMDB = New_RF_Classifier_Yelp.score(New_TFIDF_Train, Label_IMDB_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_IMDB = New_RF_Classifier_Yelp.score(New_TFIDF_Test, Label_IMDB_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train : {}".format(Skor_Train_IMDB))
print("Skor Random Forest Test  : {}".format(Skor_Test_IMDB))
print("----------------------------------------------------\n\n")

RFClassifier_predict = New_RF_Classifier_Yelp.predict(New_TFIDF_Train)
Confusion_matrix = confusion_matrix(Label_IMDB_Train, RFClassifier_predict)
print(Confusion_matrix)
print(metrics.classification_report(Label_IMDB_Train, RFClassifier_predict))

print("----------------------------------------------------\n\n")
RFClassifier_predict = New_RF_Classifier_Yelp.predict(New_TFIDF_Test)
Confusion_matrix = confusion_matrix(Label_IMDB_Test, RFClassifier_predict)
print(Confusion_matrix)
print(metrics.classification_report(Label_IMDB_Test, RFClassifier_predict))

waktu prediksi (train): 0.646s
waktu prediksi (test): 0.804s

Skor Random Forest Train : 0.992
Skor Random Forest Test  : 0.7448
----------------------------------------------------


[[499   1]
 [  7 493]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       500
           1       1.00      0.99      0.99       500

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

----------------------------------------------------


[[460 165]
 [154 471]]
              precision    recall  f1-score   support

           0       0.75      0.74      0.74       625
           1       0.74      0.75      0.75       625

    accuracy                           0.74      1250
   macro avg       0.74      0.74      0.74      1250
weighted avg       0.74      0.74      0.74      1250

